In [1]:
import pandas as pd
from io import BytesIO
import requests
import bs4
import gspread
from datetime import datetime
import sqlite3
import time

In [2]:
# python-read-sheet@read-sheets.iam.gserviceaccount.com
google_account = gspread.service_account(filename = 'credentials.json')

# In Game

In [ ]:
# https://docs.google.com/spreadsheets/d/id
zhuxian = id1 # 主线任务对话
renwuqipao = id2 # 任务气泡
sanxuanqipao = id3 # 三选气泡
...

keylist = [zhuxian, renwuqipao, sanxuanqipao]
keynamelist = ['主线任务对话', '任务气泡', '三选气泡']
print("配置表总数：",len(keylist))
for num in range(len(keylist)):
    url = 'https://docs.google.com/spreadsheets/d/' + keylist[num]
    print("{}：{}".format(keynamelist[num], url))

In [5]:
def read_spreadsheet(key):
    response = google_account.open_by_key(key)
    if key == zhuxian:
        file_name = '主线任务对话'
    elif key == renwuqipao:
        file_name = '任务气泡'
    elif key == sanxuanqipao:
        file_name = '三选气泡'        
    try:
        sheet = response.worksheet("#总表")
        print("总表")
        sheet = sheet.get_all_records()
        df = pd.DataFrame(sheet)
        df.insert(loc=0, column='sheet_name', value='总表')
        df.insert(loc=0, column='file_name', value=str(file_name))
    except:
        df = pd.DataFrame()
        length = len(response.worksheets())
        print("numbers of Sheets: {}".format(length))
        for num in range(length):
            sheet = response.get_worksheet(num)
            try:
                sheet_df = pd.DataFrame(sheet.get_all_records())
                if len(sheet_df.columns) > 15:
                    sheet_df.insert(loc=0, column='sheet_name', value=str(sheet.title))
                    sheet_df.insert(loc=0, column='file_name', value=str(file_name))
                    # df = df.append(sheet_df)
                    df = pd.concat([df, sheet_df]) # append to concat
            except:
                pass
    df.columns = df.columns.str.lower()
    print("Length: {}".format(len(df)))    
    df = df.reset_index(drop = True)
    return df

def set_columns(df):
    df.columns = df.columns.str.lower().to_list()
    df = df[['file_name', 'sheet_name', 'id', 'key', 'en', 'sc', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'indo', 'th']]
    df.columns = ['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'indo', 'th']
    return df

def drop_trash(df):
    df = df.drop(df[(df['text_id']=='long') | (df['text_id']=='程序用') | (df['text_id']=='程序用ID') | (df['text_id']=='程序用家具ID') | (df['text_id']=='string') | (df['text_id']=='int')].index)
    df = df.drop(df[(df['en']=='英语') | (df['en']==' ')].index) # en = 英语 
    df = df.drop(df[df['en'].str.len()==0].index)
    return df

In [6]:
# 主线任务对话
zhuxian_spread = read_spreadsheet(zhuxian)
zhuxian_spread = set_columns(zhuxian_spread)
zhuxian_spread = drop_trash(zhuxian_spread)
print(len(zhuxian_spread))
# time.sleep(60)
zhuxian_spread.head(2)

numbers of Sheets: 14
Length: 5185
5144


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
5,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_1,"When you told me you inherited this mansion, I...",当你告诉我你继承了这座庄园时，我认为这简直是我听过最好的消息。但……,あなたがチャールズ叔父さんからこの屋敷を受け継いたって聞いたとき、信じられなかったわ。でも…,"Als du mir erzählt hast, dass du diese Villa g...",Quand tu m’as annoncé que tu avais hérité de c...,你上次跟我說你繼承了一棟豪宅，我本以爲這是個天大的好事，但...,Quando mi hai detto che avevi ereditato questa...,"Мне не верилось, что тебе правда достался этот...",Cuando me contaste que habías heredado la mans...,"Quando você me disse que herdou essa mansão, e...","네가 이 저택을 물려받았다고 했을 때는, 너무 꿈같은 얘기라 믿어지지 않았어. 하지...",Bu köşkün sana miras kaldığını söylediğinde bu...,عندما أخبرتني أنك ورثت هذا القصر، اعتقدت أنه م...,Waktu kamu cerita bahwa kamu mewarisi rumah in...,ตอนที่เธอบอกฉันว่าได้รับคฤหาสน์นี้เป็นมรดก ฉัน...
6,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_2,"But don't worry! As your dearest friend, I'll ...",好吧，别担心！作为你最好的朋友，我会帮助你让这座庄园焕然一新！,でも、安心して。友達として、この屋敷を蘇らせるお手伝いをしてあげるから。,Keine Sorge! Als deine liebste Freundin werde ...,Mais ne t’en fais pas ! Je t’aiderai à rénover...,別擔心！作為你最要好的朋友，我會盡力讓你的豪宅重歸之前的輝煌！,Ma non preoccuparti! In qualità di tua cara am...,"Но ты не переживай! Как твой лучший друг, я по...","¡No te preocupes! ¡Como soy tu querida amiga, ...",Mas não se preocupe! Como sua mais querida ami...,안심해! 네 단짝인 내가 이 저택의 복원 공사를 도와줄게!,"Ama endişelenme! En yakın arkadaşın olarak, bu...",لكن لا داعي للقلق! بصفتي أعز صديقة لك، سأساعدك...,Tetapi tenang saja! Sebagai teman kesayanganmu...,แต่ไม่ต้องห่วง! ในฐานะที่เราเป็นเพื่อนซี้กัน ฉ...


In [7]:
# 任务气泡
time.sleep(60)
renwuqipao_spread = read_spreadsheet(renwuqipao)
renwuqipao_spread = set_columns(renwuqipao_spread)
renwuqipao_spread = drop_trash(renwuqipao_spread)
print(len(renwuqipao_spread))
renwuqipao_spread.head(2)

numbers of Sheets: 13
Length: 3599
3561


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
5,任务气泡,Language_TaskBubble_W1A1#广场,30550,,Much better!,这下好多了！,よし！ずっとマシになったわね。,Das ist schon viel besser!,C’est beaucoup mieux !,好多了！,Molto meglio!,Так-то лучше!,¡Mucho mejor!,Muito melhor!,훨씬 낫네!,Şimdi çok daha güzel oldu!,أفضل بكثير!,Lebih baik!,ดีขึ้นเยอะเลย!
6,任务气泡,Language_TaskBubble_W1A1#广场,30551,,Lemonade is re--,点心准备好了……,レモネードが出来――,Die Limonade ist fer...,La limonade est prê...,檸檬汁來啦——,La limonata è pro...,Лимонад гото...,La limonada está lis...,A limonada está pront...,레모네이드를 준비했--,Limonata ha--,عصير الليمون هو إعادة--,Limun itu sa--,น้ำมะนาวนั้น--


In [8]:
# 三选气泡
time.sleep(60)
sanxuanqipao_spread = read_spreadsheet(sanxuanqipao)
sanxuanqipao_spread = set_columns(sanxuanqipao_spread)
sanxuanqipao_spread = drop_trash(sanxuanqipao_spread)
print(len(sanxuanqipao_spread))
sanxuanqipao_spread.head(2)

numbers of Sheets: 10
Length: 2443
1791


,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th
622,三选气泡,Language_FurnitureBubble_W1A1#广场区三选时气泡,1010110110020004,,That color's inspired by Marie Antoinette's Pe...,这个配色的灵感来自于小特里亚农宫！,この色は、マリー・アントワネットの小トリアノン宮殿をイメージしてるの。,Diese Farbe wurde von Marie Antoinettes Palast...,Je me suis inspirée des couleurs du Petit Tria...,這顏色的靈感源於瑪麗·安托瓦妮特的小特里亞農宮呢！,Quel colore è ispirato al Petit Trianon di Mar...,Эта цветовая гамма навеяна самим Малым Трианон...,¡Ese color está inspirado en el palacio del pe...,Aquela cor é inspirada no Palácio Petit Triano...,마리 앙투아네트의 프티 트리아농 궁전에서 영감을 받은 색이야!,Bu renk Marie Antoinette'in Petit Trianon Sara...,هذا اللون مستوحى من قصر بيتي تريانون الخاص بال...,Warna itu terinspirasi Istana Petit Trianon mi...,นั่นเป็นสีสันที่ได้แรงบันดาลใจมาจากวังของพระนา...
623,三选气泡,Language_FurnitureBubble_W1A1#广场区三选时气泡,1010110110020005,,The slender legs on this are so luxe! And look...,落叶松的材质彰显奢华！,スレンダーな脚がすごくラグジュアリーね。ねえ、このカラマツ材を見て。,Die schlanken Beine hier sind so edel! Und sch...,Les pieds fins de ce meuble sont très luxueux....,這修長的腿也太豪華了！還是用落葉松木打造的呢！,Le gambe snelle sono così lussuose! E guarda c...,До чего же изящные и тонкие ножки! А эта столе...,¡Las finas patas son muy lujosas! ¡Mira esa ma...,As pernas esguias disso são um luxo! E olha es...,날씬한 다리가 정말 고급스러워! 저 낙엽송 목재 봐봐!,Bunun üzerindeki ince bacaklar çok gösterişli!...,الأرجل النحيلة لهذا فاخرة للغاية! وانظر إلى خش...,Kaki ramping ini sangat mewah! Dan lihat kayu ...,ขาที่เรียวของมันดูหรูมาก! ไหนจะไม้สนนั่นอีก!


# Merge Files

In [26]:
spread_sheet_list = [zhuxian_spread, renwuqipao_spread, sanxuanqipao_spread]
print(len(spread_sheet_list))

17


In [27]:
whole = pd.DataFrame()
for sheet in spread_sheet_list:
    # whole = whole.append(sheet)
    whole = pd.concat([whole, sheet]) # append to concat
print(len(whole))

20020


In [28]:
whole.head()

,file_name,sheet_name,text_id,key,en,sc,jp,de,fr,tc,it,ru,es,po,kr,tu,ar,indo,th,id
5,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_1,"When you told me you inherited this mansion, I...",当你告诉我你继承了这座庄园时，我认为这简直是我听过最好的消息。但……,あなたがチャールズ叔父さんからこの屋敷を受け継いたって聞いたとき、信じられなかったわ。でも…,"Als du mir erzählt hast, dass du diese Villa g...",Quand tu m’as annoncé que tu avais hérité de c...,你上次跟我說你繼承了一棟豪宅，我本以爲這是個天大的好事，但...,Quando mi hai detto che avevi ereditato questa...,"Мне не верилось, что тебе правда достался этот...",Cuando me contaste que habías heredado la mans...,"Quando você me disse que herdou essa mansão, e...","네가 이 저택을 물려받았다고 했을 때는, 너무 꿈같은 얘기라 믿어지지 않았어. 하지...",Bu köşkün sana miras kaldığını söylediğinde bu...,عندما أخبرتني أنك ورثت هذا القصر، اعتقدت أنه م...,Waktu kamu cerita bahwa kamu mewarisi rumah in...,ตอนที่เธอบอกฉันว่าได้รับคฤหาสน์นี้เป็นมรดก ฉัน...,NaN
6,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_2,"But don't worry! As your dearest friend, I'll ...",好吧，别担心！作为你最好的朋友，我会帮助你让这座庄园焕然一新！,でも、安心して。友達として、この屋敷を蘇らせるお手伝いをしてあげるから。,Keine Sorge! Als deine liebste Freundin werde ...,Mais ne t’en fais pas ! Je t’aiderai à rénover...,別擔心！作為你最要好的朋友，我會盡力讓你的豪宅重歸之前的輝煌！,Ma non preoccuparti! In qualità di tua cara am...,"Но ты не переживай! Как твой лучший друг, я по...","¡No te preocupes! ¡Como soy tu querida amiga, ...",Mas não se preocupe! Como sua mais querida ami...,안심해! 네 단짝인 내가 이 저택의 복원 공사를 도와줄게!,"Ama endişelenme! En yakın arkadaşın olarak, bu...",لكن لا داعي للقلق! بصفتي أعز صديقة لك، سأساعدك...,Tetapi tenang saja! Sebagai teman kesayanganmu...,แต่ไม่ต้องห่วง! ในฐานะที่เราเป็นเพื่อนซี้กัน ฉ...,NaN
7,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareTransition_3,Let's play a level to earn a <color=#3a9108>St...,就先从<color=#3a9108>清理垃圾</color>开始吧！,まずは、<color=#3a9108>雑草を片付け</color>ましょうか。,"Beginnen wir, indem wir <color=#3a9108>all die...",Commençons par <color=#3a9108>retirer tout ce ...,我們先<color=#3a9108>清理一下這些垃圾</color>。,Iniziamo <color=#3a9108>rimuovendo tutta quest...,Давай сначала <color=#3a9108>уберем весь этот ...,Comencemos por <color=#3a9108>recoger toda est...,Vamos começar <color=#3a9108>removendo todo es...,우선 <color=#3a9108>이 쓰레기부터 전부 치우자</color>.,<color=#3a9108>Tüm bu çöpleri kaldırarak</colo...,هيا نبدأ <color=#3a9108>بالتخلص من كل هذه النف...,Ayo kita mulai dengan <color=#3a9108> membuang...,เริ่มต้นด้วยการ<color=#3a9108>กำจัดขยะทั้งหมด<...,NaN
8,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareRemoveJunk_1,This isn't public property! Get out!,嘿！这儿是私人住所！谁让你们进来的？,おい！ここは所有地じゃぞ！出て行け！,Das hier ist kein öffentliches Gelände! Versch...,C’est privé ici ! Allez-vous-en !,啊！出去出去！這裡可不是什麽公共場所！,"Questa è proprietà privata, andate via!","Эй, это частная собственность! Проваливайте!",¡Esto es propiedad privada! ¡Aléjense!,Isso não é propriedade pública! Saia daqui!,여기 사유지입니다! 나가요!,Burası özel bir mülk! Defolun buradan!,هذه ليست ملكيةً عامةً! اخرجوا!,Ini bukan milik umum! Keluar!,ที่นี่ไม่ใช่ที่สาธารณะ! ออกไปซะ!,NaN
9,主线任务对话,Language_TaskDialog_W1A1#广场区对话,,T_SquareRemoveJunk_2,"Oh my goodness, Uncle Alan! It's me, Alice and--",哇，艾伦叔叔！是我啊，爱丽丝！还有这位是--,まあ、アラン叔父さん！私よ、アリスと…,"Ach du meine Güte! Onkel Alan! Ich bin‘s, Alic...","Oh mon dieu, oncle Alain ! C’est moi, Alice ! ...",天啊，阿朗叔叔！是我、艾莉絲還有～,"Oh, cielo, zio Alan! Sono io, Alice, e...","Надо же, дядя Алан! Это я, Алиса, и...","Oh Dios mío, ¡tío Alan! Soy yo, Alice y...","Meu deus, Tio Alan! Sou eu, a Alice e...","세상에, 앨런 아저씨! 저 앨리스예요. 이쪽은--","Tanrım, Alan Amca! Alice ben, hatırlamadın mı?...",يا إلهي، العم ألن! إنه أنا «أليس»، و--,"Ya Tuhan, Paman Alan! Ini aku, Alice dan--",โอ้แย่แล้ว ลุงอลันคะ! นี่หนูเอง อลิซกับ--,NaN


In [29]:
whole.columns

Index(['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de',
       'fr', 'tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'indo', 'th',
       'id'],
      dtype='object')

In [30]:
whole = whole[['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de','fr', 'tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'indo', 'th']]
whole.columns = ['file_name', 'sheet_name', 'text_id', 'key', 'en', 'sc', 'jp', 'de', 'fr','tc', 'it', 'ru', 'es', 'po', 'kr', 'tu', 'ar', 'id', 'th']

In [31]:
con = sqlite3.connect("texts.sqlite")
c = con.cursor()
c.execute("drop table nova_texts")
c.execute("create table nova_texts(file_name text, sheet_name text, text_id text, key text, en text, sc text, jp text, de text, fr text, tc text, it text, ru text, es text, po text, kr text, tu text, ar text, id text, th text)")
con.commit()
whole.to_sql("nova_texts", con, if_exists="append",index=False)
con.close()

# Historical Translation Records

In [ ]:
# whole：https://drive.google.com/drive/folders/1...
# 已翻译过的内容：https://docs.google.com/spreadsheets/d/1...

In [7]:
# https://docs.google.com/spreadsheets/d/key
key = key

In [8]:
response = google_account.open_by_key(key)
sheet = response.get_worksheet(1)
sheet_df = pd.DataFrame(sheet.get_all_records())
df = sheet_df.copy()

In [9]:
len(sheet_df), len(df)

(1473, 1473)

In [10]:
print("sheet_df Length: {}".format(len(sheet_df)))
print("df Length: {}".format(len(df)))

sheet_df Length: 1473
df Length: 1473


In [11]:
df['word_count'] = df['英语'].str.split().str.len()

In [12]:
df.columns

Index(['FileName', 'Link', 'ID', 'Note', '英语', 'MAX LENGTH', 'WORD COUNT',
       '简体中文', '日语', '德语', '法语', '繁体中文', '意大利语', '俄罗斯语', '西班牙语', '葡萄牙语', '韩语',
       '土耳其语', '阿拉伯语', '印尼语', '泰语', 'word_count'],
      dtype='object')

In [13]:
len(df.columns)

22

In [14]:
df = df[['FileName', 'Note', '英语', '简体中文', '日语', '德语', '法语', '繁体中文', '意大利语', '俄罗斯语', '西班牙语', '葡萄牙语', '韩语','土耳其语', '阿拉伯语', '印尼语', '泰语', 'Link', 'ID', 'MAX LENGTH', 'word_count']]
df.columns = ['FileName', 'Note', '英语', '简中', '日语', '德语', '法语', '繁中', '意语', '俄语', '西语', '葡语', '韩语','土语', '阿语', '印尼语', '泰语','Link', 'ID','max_length', 'word_count']

In [15]:
con = sqlite3.connect("texts.sqlite")
c = con.cursor()
# c.execute("drop table texts_translated")
# c.execute("create table texts_translated(FileName text, Note text, 英语 text, 简中 text, 日语 text, 德语 text, 法语 text, 繁中 text, 意语 text, 俄语 text, 西语 text, 葡语 text, 韩语 text, 土语 text, 阿语 text, 印尼语 text, 泰语 text , Link text, ID text, max_length text, word_count text)")
# con.commit()
df.to_sql("texts_translated", con, if_exists="append",index=False)
con.close()